# Reference snowpits

This notebook aims at extracting and plotting the data obtained at each snowpit.

## Imports and functions

In [1]:
import dask.array
from datetime import datetime

In [2]:
%run little_awk_functions.py
%run parameters.py

In get_met_forcing() - Warning: check the format of your file corresponds to the indices given in the functions (wind speed 5, surface temperature 7, time 0).


In [3]:
def simulate_new_snow_ro(nb_iterations, end_accumulation_times, start_accumulation, end_accumulation, dt, met_temp_data=[None], met_wind_data=[None], met_time_data=[0]):
    '''
    Function that computes the new snow density at each detected accumulation
    Args:
        nb_iterations: number of iterations in the simulation
        end_accumulation_times: list of ending times of accumulations in seconds since data starting date
        start_accumulation: list of the indices of starting times of accumulations in the dataset
        end_accumulation: list of the indices of ending times of accumulations in the dataset
        dt: timestep between each iteration (s)
        met_temp_data: array containing surface temperatures for each meteorological timestamp (degrees Celcius),
                default [None], i.e. use tsfc=tsfc_default
        met_wind_data: array containing wind speed for each meteorological timestamp (m.s^-1), default [None], i.e. use new_snow_ro=150
        met_time_data: array containing the meteorological timestamps (s since start of simulation), default [0]
    Returns:
        accumulation_new_snow_ro: list containing the new snow density (kg.m^-3) corresponding to each detected accumulation
    '''
    
    # Initialize new snow density
    if met_wind_data[0] != None and float(met_wind_data[0]) > 6:
        new_snow_ro = 250
    else:
        new_snow_ro = 150

    # Initialize indices of next accumulation/erosion events coming up (updated when their time is past)
    accumulation_index, wind_index = 0, 0
    accumulation_new_snow_ro = []

    for i in range(nb_iterations):
        
        # Update new snow density if needed
        if wind_index<len(met_wind_data) and met_time_data[wind_index]!=None and i*dt>=met_time_data[wind_index]:
            if met_wind_data[wind_index] != None:
                if float(met_wind_data[wind_index]) > 6:
                    new_snow_ro = 250
            else:
                new_snow_ro = 150
            wind_index += 1

        # Detection of accumulations
        if accumulation_index<len(end_accumulation_times) and i*dt>=end_accumulation_times[accumulation_index]:     # if an accumulation was passed
            accumulation_new_snow_ro.append(new_snow_ro)
            accumulation_index += 1     # next accumulation that will come up
        
    return(accumulation_new_snow_ro)

## Create dataset

In [4]:
sp1 = xr.open_dataset('pit_5_2023.nc')


In [5]:
# Pre-processing to get clean data

sp1 = sp1.ffill(dim='time')

median_space_filtering(sp1, 5, x_span=7, y_span=7)
median_time_filtering(sp1, 11)

# Add the summer surface to the dataset

define_summer_surface(sp1, start_summer_surface, end_summer_surface)

# Redefine the zero of the dataset

sp1['snow_depth'] = sp1['snow_surface'] - sp1['summer_surface']


---> Median filtering in space with a window [7, 7]
---> Median filtering in time with a window of 11
---> Summer surface defined based on scans from 2022-10-07 to 2022-10-15


## Get events' timing and plot it

In [6]:
# Define dates

data_starting_date_in_ns = float(sp1.time.values[0])

# Define the number of iterations based on duration of the data

data_starting_date_in_s = pd.to_datetime(sp1.time.values[0]).timestamp()
data_ending_date_in_s = pd.to_datetime(sp1.time.values[-1]).timestamp()
data_duration_in_s = data_ending_date_in_s - data_starting_date_in_s
nb_iterations = int(data_duration_in_s/dt + 1)

In [7]:
results_sp1 = get_snow_events(sp1, x_isel, y_isel, time_window_std, std_threshold)

start_accumulation = results_sp1[0]
start_erosion = results_sp1[1]
end_accumulation = results_sp1[2]
end_erosion = results_sp1[3]

# Convert end times into more manageable orders of magnitude

end_accumulation_times = sp1.snow_depth.isel(x=x_isel, y=y_isel, time=end_accumulation)
end_accumulation_times = (pd.to_datetime(end_accumulation_times.time).astype(int) - data_starting_date_in_ns) / 1000000000  # in s

## Get snow height for each event, convert to swe

In [8]:
accumulation_new_snow_ro = simulate_new_snow_ro(nb_iterations, end_accumulation_times, start_accumulation, end_accumulation, dt, met_temp_data=met_temp, met_wind_data=met_wind, met_time_data=met_time)

In [9]:
ro_water = 1000      # in kg per m**3

sp1_accumulation_heights = []
sp1_accumulation_duration = []
sp1_swe = []

for index in range(len(start_accumulation)):
    sp1_accumulation_heights.append(get_change_in_snow_depth(sp1, start_accumulation, end_accumulation, index, x_isel, y_isel) * 1000 / 0.01)  # in mm per m**2
    
    start = sp1.time.values[start_accumulation[index]]
    end = sp1.time.values[end_accumulation[index]]
    diff = float(end-start) / 1000000000     # in s
    sp1_accumulation_duration.append(diff)
    
    ro_layer = accumulation_new_snow_ro[index]      # in kg per m**3
    swe = sp1_accumulation_heights[index] * (ro_water/ro_layer) / sp1_accumulation_duration[index]     # in mm per s per m**2
    sp1_swe.append(swe)


## Make dataset (swe per hour) and save files

In [11]:
start = pd.to_datetime(sp1.time.values[0])
end = pd.to_datetime(sp1.time.values[-1])
nb_of_hours_in_ds = (end - start).days * 24

In [12]:
data_ds_sp1 = np.zeros(nb_of_hours_in_ds)
data_start_date = start

for event_index in range(len(start_accumulation)):
    starting_time = sp1.time.values[start_accumulation[event_index]]
    ending_time = sp1.time.values[end_accumulation[event_index]]
    ending_time_duration = sp1.time.values[end_accumulation[event_index]+1]

    starting_duration = float((starting_time-data_start_date).total_seconds()) / 3600    # in h
    starting_index = int(starting_duration // 1)

    ending_duration_true_duration = float((ending_time_duration-data_start_date).total_seconds()) / 3600    # in h
    ending_duration = float((ending_time-data_start_date).total_seconds()) / 3600    # in h
    ending_index = int(ending_duration // 1)

    starting_swe = sp1_swe[event_index] * (starting_duration - starting_index)    # percentage of the hour during which it actually precipitated
    ending_swe = sp1_swe[event_index] * (ending_duration_true_duration - ending_index)

    data_ds_sp1[starting_index] += starting_swe
    data_ds_sp1[ending_index] += ending_swe

    for i in range(starting_index+1, ending_index):
        data_ds_sp1[i] += sp1_swe[event_index]


In [13]:
times = pd.date_range(start='2022-10-07',freq='1H',periods=nb_of_hours_in_ds)

# create dataset
ds_sp1 = xr.Dataset({
    'swe': xr.DataArray(
                data   = data_ds_sp1,
                dims   = ['time'],
                coords = {'time': times},
                attrs  = {
                    'units'     : 'mm/s/m**2'
                    }
                )
            }
    )


In [ ]:
# ds_sp1.to_netcdf('snow_pit_5_precip.nc')